In [15]:
import os

os.environ["OPENAI_API_KEY"] = "key"

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "key"
os.environ["LANGCHAIN_PROJECT"] = "id"

In [16]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

In [17]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
    """Returns the input as all lower case."""
    return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Returns a random number between 0-100. input the word 'random'"""
    return random.randint(0, 100)

tools = [to_lower_case,random_number_maker]

In [19]:
from langgraph.prebuilt.tool_executor import ToolExecutor

tool_executor = ToolExecutor(tools)

from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

functions

[{'name': 'lower_case',
  'description': 'Returns the input as all lower case.',
  'parameters': {'type': 'object',
   'properties': {'input': {'type': 'string'}},
   'required': ['input']}},
 {'name': 'random_number',
  'description': "Returns a random number between 0-100. input the word 'random'",
  'parameters': {'type': 'object',
   'properties': {'input': {'type': 'string'}},
   'required': ['input']}}]

In [20]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [21]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# Set the entrypoint as `agent` where we start
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END
    }
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [22]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': 'random'}
The tool result is: 19
The agent action is tool='lower_case' tool_input={'input': 'nineteen'}
The tool result is: nineteen


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='give me a random number and then write in words and make it lower case'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_611b667b19'}, id='run-98e4dba1-2b41-4719-97d4-61228dc4fb78-0'),
  FunctionMessage(content='19', name='random_number'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"nineteen"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050'}, id='run-0a83d224-08b7-423d-b3e6-3976b27c8b54-0'),
  FunctionMessage(content='nineteen', name='lower_case'),
  AIMessage(content='The random number is 19, which is written in words as "nineteen" in lower case: **ninetee

In [24]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
# user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content='what is a Merlion?'),
  AIMessage(content='The Merlion is a famous statue and symbol of Singapore. It features a mythical creature with the head of a lion and the body of a fish. The lion\'s head represents Singapore\'s original name, Singapura, which means "Lion City" in Malay, while the fish body symbolizes Singapore\'s origins as a fishing village. The Merlion is often depicted spouting water from its mouth and is a popular tourist attraction, located at Merlion Park near Marina Bay. The statue stands at about 8.6 meters (28 feet) tall and is a significant cultural icon representing Singapore\'s history and identity.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f'}, id='run-cb686481-31a4-4ac6-9872-fd9171527daa-0')]}

In [23]:
from langchain_core.messages import HumanMessage, SystemMessage
# inputs = {"input": "give me a random number and then write in words and make it lower case", "chat_history": []}

system_message = SystemMessage(content="you are a helpful assistant")
# user_01 = HumanMessage(content="give me a random number and then write in words and make it lower case")
user_01 = HumanMessage(content="plear write 'Merlion' in lower case")
# user_01 = HumanMessage(content="what is a Merlion?")

inputs = {"messages": [system_message,user_01]}

app.invoke(inputs)

The agent action is tool='lower_case' tool_input={'input': 'Merlion'}
The tool result is: merlion


{'messages': [SystemMessage(content='you are a helpful assistant'),
  HumanMessage(content="plear write 'Merlion' in lower case"),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Merlion"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_661538dc1f'}, id='run-d3059274-2ab2-4044-8261-e75800ae18ca-0'),
  FunctionMessage(content='merlion', name='lower_case'),
  AIMessage(content='The word "Merlion" in lower case is "merlion".', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050'}, id='run-792398b4-cb6f-48e2-a3e9-f1b9caf0ecca-0')]}